# Airfoil's Aerodynamic Coefficient Prediction using ANNs

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
# importing the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
# reading the csv file
df = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA5D_15/NACA5D_15.csv")

In [4]:
# printing the first 10 rows of the dataset
df.head(10)

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL12,yL13,yL14,yL15,alpha,M,Re,CL,CD,Cm
0,5,0.0,5,0,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,...,-0.008375,-0.005206,-0.002698,-0.001083,-10.0,0.3,100000,-1.189,0.02909,0.005
1,5,0.0,5,0,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,...,-0.008375,-0.005206,-0.002698,-0.001083,-9.0,0.3,100000,-1.071,0.02203,0.005
2,5,0.0,5,0,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,...,-0.008375,-0.005206,-0.002698,-0.001083,-8.0,0.3,100000,-0.953,0.01969,0.004
3,5,0.0,5,0,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,...,-0.008375,-0.005206,-0.002698,-0.001083,-7.0,0.3,100000,-0.835,0.01629,0.004
4,5,0.0,5,0,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,...,-0.008375,-0.005206,-0.002698,-0.001083,-6.0,0.3,100000,-0.716,0.03687,0.003
5,5,0.0,5,0,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,...,-0.008375,-0.005206,-0.002698,-0.001083,-5.0,0.3,100000,-0.597,0.01921,0.003
6,5,0.0,5,0,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,...,-0.008375,-0.005206,-0.002698,-0.001083,-4.0,0.3,100000,-0.478,0.01259,0.002
7,5,0.0,5,0,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,...,-0.008375,-0.005206,-0.002698,-0.001083,-3.0,0.3,100000,-0.359,0.00917,0.002
8,5,0.0,5,0,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,...,-0.008375,-0.005206,-0.002698,-0.001083,-2.0,0.3,100000,-0.239,0.01397,0.001
9,5,0.0,5,0,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,...,-0.008375,-0.005206,-0.002698,-0.001083,-1.0,0.3,100000,-0.120,0.01360,0.001


In [5]:
# checking for missing values in the dataset
df.isnull().sum()

t            0
CL_design    0
p            0
s            0
yU1          0
yU2          0
yU3          0
yU4          0
yU5          0
yU6          0
yU7          0
yU8          0
yU9          0
yU10         0
yU11         0
yU12         0
yU13         0
yU14         0
yU15         0
yL1          0
yL2          0
yL3          0
yL4          0
yL5          0
yL6          0
yL7          0
yL8          0
yL9          0
yL10         0
yL11         0
yL12         0
yL13         0
yL14         0
yL15         0
alpha        0
M            0
Re           0
CL           0
CD           0
Cm           0
dtype: int64

In [6]:
# description of the dataset
df.describe()

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL12,yL13,yL14,yL15,alpha,M,Re,CL,CD,Cm
count,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,...,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.00000,236880.000000,236880.000000,236880.000000,236880.000000
mean,19.960106,0.841223,27.632979,0.493351,0.047538,0.078390,0.104199,0.123085,0.132926,0.133584,...,-0.031123,-0.020091,-0.010633,-0.004201,0.000000,0.20000,300000.000000,0.219178,0.025908,0.008664
std,10.003265,0.595420,16.860962,0.499957,0.030669,0.041056,0.050985,0.059541,0.065274,0.067362,...,0.021509,0.013390,0.006790,0.002508,6.055313,0.08165,141421.654747,0.925885,0.011283,0.092522
min,5.000000,0.000000,5.000000,0.000000,0.003227,0.000664,-0.007350,-0.018600,-0.030529,-0.040678,...,-0.114190,-0.075561,-0.039293,-0.013004,-10.000000,0.10000,100000.000000,-2.607000,0.002690,-0.169000
25%,10.000000,0.200000,15.000000,0.000000,0.022022,0.047241,0.066107,0.078454,0.082814,0.082163,...,-0.045833,-0.029399,-0.015450,-0.006087,-5.000000,0.10000,200000.000000,-0.485000,0.018250,-0.031000
50%,20.000000,0.800000,25.000000,0.000000,0.041569,0.075667,0.103424,0.122582,0.131741,0.132364,...,-0.030523,-0.019715,-0.010449,-0.004242,-0.000000,0.20000,300000.000000,0.218000,0.024650,-0.004000
75%,30.000000,1.400000,45.000000,1.000000,0.067140,0.105613,0.137142,0.162374,0.176566,0.178766,...,-0.014689,-0.010201,-0.005396,-0.002194,5.000000,0.30000,400000.000000,0.928000,0.031580,0.018000
max,35.000000,1.800000,75.000000,1.000000,0.140028,0.198915,0.254883,0.298955,0.326478,0.333466,...,0.019131,0.010615,0.004440,0.004549,10.000000,0.30000,500000.000000,3.020000,0.617020,0.453000


In [7]:
# shuffling all the rows
df = df.sample(frac=1)
df.head(10)

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL12,yL13,yL14,yL15,alpha,M,Re,CL,CD,Cm
43372,25,0.2,15,1,0.047178,0.078507,0.104923,0.123890,0.133972,0.135590,...,-0.040830,-0.025555,-0.013319,-0.005376,-3.0,0.3,100000,-0.328,0.02769,0.013
86361,25,0.6,5,1,0.065469,0.094436,0.112452,0.128158,0.133963,0.129835,...,-0.059271,-0.037840,-0.019476,-0.007017,-1.0,0.2,300000,-0.482,0.01210,0.154
76233,25,0.4,35,1,0.047442,0.081046,0.110993,0.134693,0.150073,0.155789,...,-0.037223,-0.024537,-0.013308,-0.005458,-7.0,0.2,100000,-0.704,0.03037,-0.009
1479,25,0.0,5,0,0.034641,0.065617,0.091387,0.110451,0.121726,0.124927,...,-0.041873,-0.026030,-0.013490,-0.005416,-1.0,0.2,100000,-0.136,0.02670,0.002
11654,10,0.0,25,1,0.013871,0.026280,0.036611,0.044258,0.048779,0.050057,...,-0.016745,-0.010412,-0.005397,-0.002167,10.0,0.3,500000,1.240,0.02212,-0.009
201985,10,1.6,25,0,0.036025,0.075314,0.115936,0.148375,0.165164,0.164409,...,0.007814,0.003730,0.000993,-0.000555,-3.0,0.1,400000,0.601,0.01915,-0.121
180201,15,1.4,25,0,0.047988,0.087809,0.126923,0.158147,0.174816,0.175321,...,-0.003416,-0.003095,-0.002414,-0.001793,-10.0,0.1,200000,-0.363,0.03141,-0.106
32096,20,0.0,75,0,0.027713,0.052494,0.073109,0.088361,0.097381,0.099942,...,-0.033498,-0.020824,-0.010792,-0.004333,-2.0,0.3,400000,-0.269,0.01989,0.003
104543,35,0.6,45,1,0.050863,0.094362,0.130652,0.157672,0.173870,0.178764,...,-0.056344,-0.034805,-0.017763,-0.006790,-5.0,0.2,400000,-0.671,0.03181,0.002
6726,5,0.0,15,1,0.006940,0.013159,0.018336,0.022158,0.024408,0.025038,...,-0.008370,-0.005204,-0.002697,-0.001083,-4.0,0.2,100000,-0.465,0.01138,0.002


In [8]:
# dropping columns FoilID, m, p & t
df = df.drop(columns=['t', 'CL_design', 'p', 's'], axis=1)
df.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL12,yL13,yL14,yL15,alpha,M,Re,CL,CD,Cm
43372,0.047178,0.078507,0.104923,0.123890,0.133972,0.135590,0.129512,0.117211,0.100554,0.081482,...,-0.040830,-0.025555,-0.013319,-0.005376,-3.0,0.3,100000,-0.328,0.02769,0.013
86361,0.065469,0.094436,0.112452,0.128158,0.133963,0.129835,0.117288,0.099059,0.078347,0.058010,...,-0.059271,-0.037840,-0.019476,-0.007017,-1.0,0.2,300000,-0.482,0.01210,0.154
76233,0.047442,0.081046,0.110993,0.134693,0.150073,0.155789,0.151611,0.138556,0.118732,0.094900,...,-0.037223,-0.024537,-0.013308,-0.005458,-7.0,0.2,100000,-0.704,0.03037,-0.009
1479,0.034641,0.065617,0.091387,0.110451,0.121726,0.124927,0.120670,0.110292,0.095472,0.078052,...,-0.041873,-0.026030,-0.013490,-0.005416,-1.0,0.2,100000,-0.136,0.02670,0.002
11654,0.013871,0.026280,0.036611,0.044258,0.048779,0.050057,0.048341,0.044171,0.038226,0.031243,...,-0.016745,-0.010412,-0.005397,-0.002167,10.0,0.3,500000,1.240,0.02212,-0.009
201985,0.036025,0.075314,0.115936,0.148375,0.165164,0.164409,0.149472,0.129053,0.106743,0.083918,...,0.007814,0.003730,0.000993,-0.000555,-3.0,0.1,400000,0.601,0.01915,-0.121
180201,0.047988,0.087809,0.126923,0.158147,0.174816,0.175321,0.161724,0.141405,0.118192,0.093781,...,-0.003416,-0.003095,-0.002414,-0.001793,-10.0,0.1,200000,-0.363,0.03141,-0.106
32096,0.027713,0.052494,0.073109,0.088361,0.097381,0.099942,0.096536,0.088234,0.076377,0.062442,...,-0.033498,-0.020824,-0.010792,-0.004333,-2.0,0.3,400000,-0.269,0.01989,0.003
104543,0.050863,0.094362,0.130652,0.157672,0.173870,0.178764,0.173098,0.158638,0.137689,0.112846,...,-0.056344,-0.034805,-0.017763,-0.006790,-5.0,0.2,400000,-0.671,0.03181,0.002
6726,0.006940,0.013159,0.018336,0.022158,0.024408,0.025038,0.024177,0.022091,0.019118,0.015626,...,-0.008370,-0.005204,-0.002697,-0.001083,-4.0,0.2,100000,-0.465,0.01138,0.002


In [9]:
# defining the feature and target columns
X = df.drop(columns=['CL', 'CD', 'Cm'])
y = df[['CL', 'CD', 'Cm']]

In [10]:
# displaying the features table
X.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL9,yL10,yL11,yL12,yL13,yL14,yL15,alpha,M,Re
43372,0.047178,0.078507,0.104923,0.123890,0.133972,0.135590,0.129512,0.117211,0.100554,0.081482,...,-0.090628,-0.074826,-0.057735,-0.040830,-0.025555,-0.013319,-0.005376,-3.0,0.3,100000
86361,0.065469,0.094436,0.112452,0.128158,0.133963,0.129835,0.117288,0.099059,0.078347,0.058010,...,-0.115056,-0.100117,-0.080869,-0.059271,-0.037840,-0.019476,-0.007017,-1.0,0.2,300000
76233,0.047442,0.081046,0.110993,0.134693,0.150073,0.155789,0.151611,0.138556,0.118732,0.094900,...,-0.072757,-0.061803,-0.049868,-0.037223,-0.024537,-0.013308,-0.005458,-7.0,0.2,100000
1479,0.034641,0.065617,0.091387,0.110451,0.121726,0.124927,0.120670,0.110292,0.095472,0.078052,...,-0.095472,-0.078052,-0.059684,-0.041873,-0.026030,-0.013490,-0.005416,-1.0,0.2,100000
11654,0.013871,0.026280,0.036611,0.044258,0.048779,0.050057,0.048341,0.044171,0.038226,0.031243,...,-0.038152,-0.031199,-0.023863,-0.016745,-0.010412,-0.005397,-0.002167,10.0,0.3,500000
201985,0.036025,0.075314,0.115936,0.148375,0.165164,0.164409,0.149472,0.129053,0.106743,0.083918,...,0.029167,0.020478,0.013373,0.007814,0.003730,0.000993,-0.000555,-3.0,0.1,400000
180201,0.047988,0.087809,0.126923,0.158147,0.174816,0.175321,0.161724,0.141405,0.118192,0.093781,...,0.001988,-0.001277,-0.002945,-0.003416,-0.003095,-0.002414,-0.001793,-10.0,0.1,200000
32096,0.027713,0.052494,0.073109,0.088361,0.097381,0.099942,0.096536,0.088234,0.076377,0.062442,...,-0.076377,-0.062442,-0.047747,-0.033498,-0.020824,-0.010792,-0.004333,-2.0,0.3,400000
104543,0.050863,0.094362,0.130652,0.157672,0.173870,0.178764,0.173098,0.158638,0.137689,0.112846,...,-0.129622,-0.105691,-0.080598,-0.056344,-0.034805,-0.017763,-0.006790,-5.0,0.2,400000
6726,0.006940,0.013159,0.018336,0.022158,0.024408,0.025038,0.024177,0.022091,0.019118,0.015626,...,-0.019071,-0.015595,-0.011928,-0.008370,-0.005204,-0.002697,-0.001083,-4.0,0.2,100000


In [11]:
# displaying the targets table
y.head(10)

,CL,CD,Cm
43372,-0.328,0.02769,0.013
86361,-0.482,0.01210,0.154
76233,-0.704,0.03037,-0.009
1479,-0.136,0.02670,0.002
11654,1.240,0.02212,-0.009
201985,0.601,0.01915,-0.121
180201,-0.363,0.03141,-0.106
32096,-0.269,0.01989,0.003
104543,-0.671,0.03181,0.002
6726,-0.465,0.01138,0.002


## Splitting the Dataset

In [12]:
# splitting dataset into training, validation & testing dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
# normalizing the training, validation & testing datasets
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_val = X_scaler.transform(X_val)
X_test = X_scaler.transform(X_test)

In [14]:
# displaying X_train, X_val & X_test after standardization
display(pd.DataFrame(X_train).head())
display(pd.DataFrame(X_val).head())
display(pd.DataFrame(X_test).head())

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.769507,0.912298,0.933425,0.835144,0.688002,0.584103,0.532562,0.518121,0.526879,0.550836,...,0.538292,0.561599,0.588326,0.609676,0.614394,0.579291,0.380350,-1.155727,-1.224176,-1.415142
1,-0.196637,0.006047,0.104831,0.156955,0.199999,0.241412,0.289282,0.345766,0.408134,0.476784,...,-1.097051,-1.029632,-0.958011,-0.887259,-0.830376,-0.816388,-0.916168,-0.165235,0.000547,-0.707756
2,-0.964556,-0.818019,-0.580175,-0.404626,-0.326227,-0.342448,-0.405176,-0.444347,-0.456159,-0.451802,...,1.871593,1.887766,1.903002,1.907836,1.897040,1.868253,1.722646,1.155421,-1.224176,-0.707756
3,0.459651,0.487430,0.551539,0.605497,0.637494,0.644008,0.618124,0.568993,0.517258,0.465118,...,-0.243070,-0.296559,-0.347435,-0.392149,-0.431079,-0.474179,-0.550991,1.650667,-1.224176,0.707014
4,-0.727558,-0.770349,-0.741219,-0.686116,-0.618577,-0.552385,-0.491869,-0.438276,-0.390522,-0.343121,...,0.391664,0.446637,0.501366,0.550043,0.585684,0.600084,0.540387,0.990339,-1.224176,-0.000371


,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,2.088154,1.611853,1.143717,0.728440,0.423563,0.128130,-0.179100,-0.508515,-0.853681,-1.192159,...,-1.742602,-2.077436,-2.388767,-2.644338,-2.805814,-2.798736,-2.191642,0.330011,0.000547,0.707014
1,-0.773460,-0.722331,-0.558806,-0.383213,-0.223035,-0.096737,-0.010951,0.027073,0.009451,-0.068333,...,1.569404,1.489953,1.344190,1.155090,0.966242,0.823851,0.770169,-0.000153,-1.224176,-1.415142
2,1.036567,0.821556,0.693305,0.830667,0.898973,0.943267,0.985892,1.036969,1.095475,1.161150,...,-1.253962,-1.205401,-1.148006,-1.089037,-1.044291,-1.051818,-1.212115,-0.330317,-1.224176,-1.415142
3,-0.901382,-0.991117,-1.022136,-1.020149,-0.994053,-0.955830,-0.910438,-0.859229,-0.802352,-0.740484,...,-0.040059,0.030161,0.102835,0.173028,0.234638,0.284919,0.319007,-0.165235,-1.224176,-1.415142
4,-0.773460,-0.722331,-0.558806,-0.383213,-0.223035,-0.096737,-0.010951,0.027073,0.009451,-0.068333,...,1.569404,1.489953,1.344190,1.155090,0.966242,0.823851,0.770169,0.164929,0.000547,-1.415142


,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,2.191180,2.150535,2.062690,1.918909,1.737067,1.576896,1.463168,1.374791,1.296003,1.225159,...,-0.309995,-0.389028,-0.456398,-0.512514,-0.563574,-0.633462,-0.801319,-0.990645,0.000547,1.414399
1,-0.304263,-0.204946,-0.140674,-0.090674,-0.035667,0.023649,0.090424,0.164657,0.243306,0.326379,...,-0.493568,-0.407842,-0.319751,-0.229037,-0.131333,0.003260,0.333898,-0.990645,-1.224176,1.414399
2,-0.492441,-0.225338,-0.107034,-0.058322,-0.031109,-0.011111,0.012918,0.046570,0.091609,0.149683,...,-1.431811,-1.376497,-1.306708,-1.227531,-1.152604,-1.106979,-1.115399,0.164929,-1.224176,1.414399
3,-0.188590,-0.161555,-0.097289,-0.044753,-0.003700,0.012906,0.005729,-0.012228,-0.027779,-0.041242,...,-0.013517,-0.024522,-0.033636,-0.040096,-0.045625,-0.056079,-0.093288,-1.485891,-1.224176,-0.707756
4,1.072413,1.160446,1.277975,1.345926,1.377248,1.370998,1.324725,1.285720,1.276887,1.288217,...,0.515044,0.519794,0.530114,0.535665,0.521758,0.453021,0.154272,-0.495399,1.225269,-0.000371


In [15]:
# displaying y_train, y_val & y_test after standardization
display(pd.DataFrame(y_train).head())
display(pd.DataFrame(y_val).head())
display(pd.DataFrame(y_test).head())

,CL,CD,Cm
176088,-0.189,0.02814,-0.044
1710,-0.141,0.02816,0.002
157538,1.490,0.02332,-0.093
94898,1.641,0.03742,0.002
73453,0.962,0.02102,-0.042


,CL,CD,Cm
196425,-0.624,0.02056,0.383
142390,0.484,0.01612,-0.056
106583,-0.017,0.02061,0.003
53559,-0.198,0.01979,0.016
142496,0.614,0.02055,-0.057


,CL,CD,Cm
201163,-0.143,0.02024,0.036
213973,-0.668,0.02319,-0.016
101840,-0.120,0.02505,0.042
71527,-0.974,0.02611,0.019
181069,0.639,0.02377,-0.096


In [16]:
# defining the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, min_lr=0.00001)

## Network Performance Due to Multiple Hidden Layers

### Case 1: Model [64, 3]

In [17]:
# RMSE and R² metric scores on test data for Model M_64
M_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64...")

    M_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_df = pd.DataFrame(M_64_metrics)

print(results_M_64_df.mean())



Run 1/20 for Model M_64...


I0000 00:00:1764188692.098589 2913027 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6155 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1764188694.464645 2913152 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Run 2/20 for Model M_64...

Run 3/20 for Model M_64...

Run 4/20 for Model M_64...

Run 5/20 for Model M_64...

Run 6/20 for Model M_64...

Run 7/20 for Model M_64...

Run 8/20 for Model M_64...

Run 9/20 for Model M_64...

Run 10/20 for Model M_64...

Run 11/20 for Model M_64...

Run 12/20 for Model M_64...

Run 13/20 for Model M_64...

Run 14/20 for Model M_64...

Run 15/20 for Model M_64...

Run 16/20 for Model M_64...

Run 17/20 for Model M_64...

Run 18/20 for Model M_64...

Run 19/20 for Model M_64...

Run 20/20 for Model M_64...
CL_rmse    0.005831
CD_rmse    0.006601
Cm_rmse    0.002401
CL_r2      0.999960
CD_r2      0.670914
Cm_r2      0.999297
dtype: float32


### Case 2: Model [64, 32, 3]

In [18]:
# RMSE and R² metric scores on test data for Model M_64_32
M_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32...")
    
    M_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_df = pd.DataFrame(M_64_32_metrics)

print(results_M_64_32_df.mean())



Run 1/20 for Model M_64_32...

Run 2/20 for Model M_64_32...

Run 3/20 for Model M_64_32...

Run 4/20 for Model M_64_32...

Run 5/20 for Model M_64_32...

Run 6/20 for Model M_64_32...

Run 7/20 for Model M_64_32...

Run 8/20 for Model M_64_32...

Run 9/20 for Model M_64_32...

Run 10/20 for Model M_64_32...

Run 11/20 for Model M_64_32...

Run 12/20 for Model M_64_32...

Run 13/20 for Model M_64_32...

Run 14/20 for Model M_64_32...

Run 15/20 for Model M_64_32...

Run 16/20 for Model M_64_32...

Run 17/20 for Model M_64_32...

Run 18/20 for Model M_64_32...

Run 19/20 for Model M_64_32...

Run 20/20 for Model M_64_32...
CL_rmse    0.004880
CD_rmse    0.006318
Cm_rmse    0.002039
CL_r2      0.999972
CD_r2      0.698665
Cm_r2      0.999493
dtype: float32


### Case 3: Model [64, 32, 16, 3]

In [19]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")
    
    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.004593
CD_rmse    0.006250
Cm_rmse    0.001990
CL_r2      0.999975
CD_r2      0.705177
Cm_r2      0.999519
dtype: float32


### Case 4: Model [64, 32, 16, 8, 3]

In [20]:
# RMSE and R² metric scores on test data for Model M_64_32_16_8
M_64_32_16_8_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16_8...")
    
    M_64_32_16_8 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16_8.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16_8.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16_8.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_8_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_8_df = pd.DataFrame(M_64_32_16_8_metrics)

print(results_M_64_32_16_8_df.mean())



Run 1/20 for Model M_64_32_16_8...

Run 2/20 for Model M_64_32_16_8...

Run 3/20 for Model M_64_32_16_8...

Run 4/20 for Model M_64_32_16_8...

Run 5/20 for Model M_64_32_16_8...

Run 6/20 for Model M_64_32_16_8...

Run 7/20 for Model M_64_32_16_8...

Run 8/20 for Model M_64_32_16_8...

Run 9/20 for Model M_64_32_16_8...

Run 10/20 for Model M_64_32_16_8...

Run 11/20 for Model M_64_32_16_8...

Run 12/20 for Model M_64_32_16_8...

Run 13/20 for Model M_64_32_16_8...

Run 14/20 for Model M_64_32_16_8...

Run 15/20 for Model M_64_32_16_8...

Run 16/20 for Model M_64_32_16_8...

Run 17/20 for Model M_64_32_16_8...

Run 18/20 for Model M_64_32_16_8...

Run 19/20 for Model M_64_32_16_8...

Run 20/20 for Model M_64_32_16_8...
CL_rmse    0.004282
CD_rmse    0.006311
Cm_rmse    0.002023
CL_r2      0.999978
CD_r2      0.699260
Cm_r2      0.999500
dtype: float32


In [21]:
M_64_stats = results_M_64_df.mean()
M_64_32_stats = results_M_64_32_df.mean()
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_64_32_16_8_stats = results_M_64_32_16_8_df.mean()

architectures = [
    '64, 3',
    '64, 32, 3',
    '64, 32, 16, 3',
    '64, 32, 16, 8, 3'
]

table_data = [
    (M_64_stats['CL_rmse'], M_64_stats['CD_rmse'], M_64_stats['Cm_rmse'],
     M_64_stats['CL_r2'], M_64_stats['CD_r2'], M_64_stats['Cm_r2']),
    
    (M_64_32_stats['CL_rmse'], M_64_32_stats['CD_rmse'], M_64_32_stats['Cm_rmse'],
     M_64_32_stats['CL_r2'], M_64_32_stats['CD_r2'], M_64_32_stats['Cm_r2']),
    
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_64_32_16_8_stats['CL_rmse'], M_64_32_16_8_stats['CD_rmse'], M_64_32_16_8_stats['Cm_rmse'],
     M_64_32_16_8_stats['CL_r2'], M_64_32_16_8_stats['CD_r2'], M_64_32_16_8_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 5), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))


## Network Performance Due to Increasing Neurons

### Case 1: Model [64, 32, 16, 3]

In [22]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")

    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.004566
CD_rmse    0.006282
Cm_rmse    0.002029
CL_r2      0.999975
CD_r2      0.701907
Cm_r2      0.999498
dtype: float32


### Case 2: Model [128, 64, 32, 3]

In [23]:
# RMSE and R² metric scores on test data for Model M_128_64_32
M_128_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_128_64_32...")

    M_128_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_128_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_128_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_128_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_128_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_128_64_32_df = pd.DataFrame(M_128_64_32_metrics)

print(results_M_128_64_32_df.mean())



Run 1/20 for Model M_128_64_32...

Run 2/20 for Model M_128_64_32...

Run 3/20 for Model M_128_64_32...

Run 4/20 for Model M_128_64_32...

Run 5/20 for Model M_128_64_32...

Run 6/20 for Model M_128_64_32...

Run 7/20 for Model M_128_64_32...

Run 8/20 for Model M_128_64_32...

Run 9/20 for Model M_128_64_32...

Run 10/20 for Model M_128_64_32...

Run 11/20 for Model M_128_64_32...

Run 12/20 for Model M_128_64_32...

Run 13/20 for Model M_128_64_32...

Run 14/20 for Model M_128_64_32...

Run 15/20 for Model M_128_64_32...

Run 16/20 for Model M_128_64_32...

Run 17/20 for Model M_128_64_32...

Run 18/20 for Model M_128_64_32...

Run 19/20 for Model M_128_64_32...

Run 20/20 for Model M_128_64_32...
CL_rmse    0.003467
CD_rmse    0.005818
Cm_rmse    0.001506
CL_r2      0.999986
CD_r2      0.744199
Cm_r2      0.999724
dtype: float32


### Case 3: Model [256, 128, 64, 3]

In [24]:
# RMSE and R² metric scores on test data for Model M_256_128_64
M_256_128_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_256_128_64...")

    M_256_128_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_256_128_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_256_128_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_256_128_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_256_128_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_256_128_64_df = pd.DataFrame(M_256_128_64_metrics)

print(results_M_256_128_64_df.mean())



Run 1/20 for Model M_256_128_64...

Run 2/20 for Model M_256_128_64...

Run 3/20 for Model M_256_128_64...

Run 4/20 for Model M_256_128_64...

Run 5/20 for Model M_256_128_64...

Run 6/20 for Model M_256_128_64...

Run 7/20 for Model M_256_128_64...

Run 8/20 for Model M_256_128_64...

Run 9/20 for Model M_256_128_64...

Run 10/20 for Model M_256_128_64...

Run 11/20 for Model M_256_128_64...

Run 12/20 for Model M_256_128_64...

Run 13/20 for Model M_256_128_64...

Run 14/20 for Model M_256_128_64...

Run 15/20 for Model M_256_128_64...

Run 16/20 for Model M_256_128_64...

Run 17/20 for Model M_256_128_64...

Run 18/20 for Model M_256_128_64...

Run 19/20 for Model M_256_128_64...

Run 20/20 for Model M_256_128_64...
CL_rmse    0.002889
CD_rmse    0.005332
Cm_rmse    0.001147
CL_r2      0.999990
CD_r2      0.785196
Cm_r2      0.999833
dtype: float32


### Case 4: Model [512, 256, 128, 3]

In [25]:
# RMSE and R² metric scores on test data for Model M_512_256_128
M_512_256_128_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_512_256_128...")

    M_512_256_128 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_512_256_128.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_512_256_128.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_512_256_128.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_512_256_128_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_512_256_128_df = pd.DataFrame(M_512_256_128_metrics)

print(results_M_512_256_128_df.mean())



Run 1/20 for Model M_512_256_128...

Run 2/20 for Model M_512_256_128...

Run 3/20 for Model M_512_256_128...

Run 4/20 for Model M_512_256_128...

Run 5/20 for Model M_512_256_128...

Run 6/20 for Model M_512_256_128...

Run 7/20 for Model M_512_256_128...

Run 8/20 for Model M_512_256_128...

Run 9/20 for Model M_512_256_128...

Run 10/20 for Model M_512_256_128...

Run 11/20 for Model M_512_256_128...

Run 12/20 for Model M_512_256_128...

Run 13/20 for Model M_512_256_128...

Run 14/20 for Model M_512_256_128...

Run 15/20 for Model M_512_256_128...

Run 16/20 for Model M_512_256_128...

Run 17/20 for Model M_512_256_128...

Run 18/20 for Model M_512_256_128...

Run 19/20 for Model M_512_256_128...

Run 20/20 for Model M_512_256_128...
CL_rmse    0.002596
CD_rmse    0.005229
Cm_rmse    0.000994
CL_r2      0.999991
CD_r2      0.792163
Cm_r2      0.999879
dtype: float32


### Case 5: Model [1024, 512, 256, 3]

In [26]:
# RMSE and R² metric scores on test data for Model M_1024_512_256
M_1024_512_256_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_1024_512_256...")

    M_1024_512_256 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_1024_512_256.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_1024_512_256.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_1024_512_256.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_1024_512_256_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_1024_512_256_df = pd.DataFrame(M_1024_512_256_metrics)

print(results_M_1024_512_256_df.mean())



Run 1/20 for Model M_1024_512_256...

Run 2/20 for Model M_1024_512_256...

Run 3/20 for Model M_1024_512_256...

Run 4/20 for Model M_1024_512_256...

Run 5/20 for Model M_1024_512_256...

Run 6/20 for Model M_1024_512_256...

Run 7/20 for Model M_1024_512_256...

Run 8/20 for Model M_1024_512_256...

Run 9/20 for Model M_1024_512_256...

Run 10/20 for Model M_1024_512_256...

Run 11/20 for Model M_1024_512_256...

Run 12/20 for Model M_1024_512_256...

Run 13/20 for Model M_1024_512_256...

Run 14/20 for Model M_1024_512_256...

Run 15/20 for Model M_1024_512_256...

Run 16/20 for Model M_1024_512_256...

Run 17/20 for Model M_1024_512_256...

Run 18/20 for Model M_1024_512_256...

Run 19/20 for Model M_1024_512_256...

Run 20/20 for Model M_1024_512_256...
CL_rmse    0.002564
CD_rmse    0.005044
Cm_rmse    0.001025
CL_r2      0.999992
CD_r2      0.807782
Cm_r2      0.999867
dtype: float32


In [27]:
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_128_64_32_stats = results_M_128_64_32_df.mean()
M_256_128_64_stats = results_M_256_128_64_df.mean()
M_512_256_128_stats = results_M_512_256_128_df.mean()
M_1024_512_256_stats = results_M_1024_512_256_df.mean()

architectures = [
    '64, 32, 16, 3',
    '128, 64, 32, 3',
    '256, 128, 64, 3',
    '512, 256, 128, 3',
    '1024, 512, 256, 3'
]

table_data = [
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_128_64_32_stats['CL_rmse'], M_128_64_32_stats['CD_rmse'], M_128_64_32_stats['Cm_rmse'],
     M_128_64_32_stats['CL_r2'], M_128_64_32_stats['CD_r2'], M_128_64_32_stats['Cm_r2']),
    
    (M_256_128_64_stats['CL_rmse'], M_256_128_64_stats['CD_rmse'], M_256_128_64_stats['Cm_rmse'],
     M_256_128_64_stats['CL_r2'], M_256_128_64_stats['CD_r2'], M_256_128_64_stats['Cm_r2']),
    
    (M_512_256_128_stats['CL_rmse'], M_512_256_128_stats['CD_rmse'], M_512_256_128_stats['Cm_rmse'],
     M_512_256_128_stats['CL_r2'], M_512_256_128_stats['CD_r2'], M_512_256_128_stats['Cm_r2']),
    
    (M_1024_512_256_stats['CL_rmse'], M_1024_512_256_stats['CD_rmse'], M_1024_512_256_stats['Cm_rmse'],
     M_1024_512_256_stats['CL_r2'], M_1024_512_256_stats['CD_r2'], M_1024_512_256_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 6), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))
